# Dynamic Programming
---
> 동적계획법 실습
1. Policy Iteration
2. Value Iteration

* D.P 는 공통적으로 모델을 알고 있는 Model-Based 환경이다
* 여기서 Model 을 알고 있다라는 것은 이란 보상함수 R(s,a), 전이확률함수 P(s'|s,a) 의 정보를 agent 가 알고 업데이트 할 때 사용한다는 의미임. 
* 엄밀히 말하면 D.P 는 Learning 이라기보다 Planning 이다.
* 전체 상태공간에 대한 가치를 한 번에 다 구한다.

여기서 테스트할 환경의 특징은
1. 4x4 grid world
2. UP, DOWN, LEFT, RIGHT
4. 목적지에 도착하면 에피소드가 종료된다.
5. 목적지에서 가치는 0 이다.

### Policy Iteration
---
>1. Policy Evaluation
2. Policy Improvement

* On-Policy method
* Bellman Expected Equation 사용
* Value 와 Policy 분리되어 있고 따로 업데이트
* 보통 '평가-발전' 순서가 있는데 1 회 평가 --> 1 회 발전 을 GPI(Generalized Policy Iteration) 이라고 한다.

In [27]:
import numpy as np

In [37]:
class Policy_Iteration:
    
    def __init__(self):
        self.n_map = 4
        self.n_action = 4
        self.gamma = 0.9
        self.grid_world = np.array([0,0,0,0,
                     0,-1,0,-1,
                     0,0,0,-1,
                    -1,0,0,1]).reshape(self.n_map,-1)
        self.value = np.zeros_like(self.grid_world)
        self.policy = np.ones([self.n_action, self.n_map, self.n_map]) / self.n_action
    
    
    '''
    Evaluation
    '''
    def Evaluation(self):
        v_tmp = np.zeros_like(self.grid_world, dtype=np.float16)
        for row in range(self.n_map):
            for col in range(self.n_map):
                for a in range(self.n_action):
                    reward = self.get_reward(a, row, col)
                    next_state = self.get_state(a, row, col)

                    # P.I 확률계산식 - bellman expected equation.
                    v_tmp[row, col] += self.policy[a, row, col] * (reward + self.gamma * self.value[next_state[0], next_state[1]])
        
        v_tmp[3,3] = 0
        self.value = v_tmp
    
    
    '''
    Imporvement
    '''
    def Improvement(self):
        for row in range(self.n_map):
            for col in range(self.n_map):
                next_p = np.zeros([4], dtype=np.float16)
                cnt = 1
                max_tmp = -9999
                idx = []
                
                for a in range(self.n_action):
                    next_state = self.get_state(a, row, col)
                    tmp = self.get_reward(a, row, col) + self.gamma * self.value[next_state[0], next_state[1]]
                    
                    if(tmp > max_tmp):
                        max_tmp = tmp
                        idx.clear()
                        cnt = 1
                        idx.append(a)
                        
                    elif(tmp == max_tmp):
                        cnt += 1
                        idx.append(a)
                        
                prob = 1.0 / cnt
                
                for i in idx:
                    next_p[i] = prob
                    
                self.policy[ : , row, col] = next_p
    
    
    '''
    R(s,a)
    P(s'|s,a)
    
    0  1  2  3
    상 하 좌 우
    '''
    def get_reward(self, a, r, c):
        
        if(a == 0 and r > 0):
            r -= 1

        elif(a == 1 and r < 3):
            r += 1

        elif(a == 2 and c > 0):
            c -= 1

        elif(a == 3 and c < 3):
            c += 1
        
        reward = self.grid_world[r, c]

        return reward

    
    '''
    next state
    '''
    def get_state(self, a, r, c):
        
        if(a == 0 and r > 0):
            r -= 1

        elif(a == 1 and r < 3):
            r += 1

        elif(a == 2 and c > 0):
            c -= 1

        elif(a == 3 and c < 3):
            c += 1
            
        return r, c
    
    def get_policy(self):
        policy = np.zeros_like(self.grid_world)
        
        for row in range(self.n_map):
            for col in range(self.n_map):
                policy[row, col] = np.argmax(self.policy[ : , row, col])
                
        return policy

In [38]:
p_i = Policy_Iteration()

In [39]:
for i in range(10):
    # GPI(Generalized Policy Improvement) - 1 Eval --> 1 Improve
    p_i.Evaluation()
    p_i.Improvement()

print(p_i.grid_world)
print(p_i.value)
print(p_i.policy)
print(p_i.get_policy())

[[ 0  0  0  0]
 [ 0 -1  0 -1]
 [ 0  0  0 -1]
 [-1  0  0  1]]
[[0.591  0.6562 0.729  0.6562]
 [0.6562 0.729  0.81   0.729 ]
 [0.729  0.81   0.9    1.    ]
 [0.81   0.9    1.     0.    ]]
[[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.5 0.  1.  0. ]
  [1.  0.5 1.  0. ]
  [0.  0.5 1.  1. ]
  [0.  0.  0.  0.5]]

 [[0.  0.  0.  1. ]
  [0.  0.  0.  1. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.5 1.  0.  0. ]
  [0.  0.5 0.  0. ]
  [1.  0.5 0.  0. ]
  [1.  1.  1.  0.5]]]
[[1 3 1 2]
 [1 1 1 2]
 [3 1 1 1]
 [3 3 3 1]]


In [31]:
class Value_Iteration:
    
    def __init__(self):
        self.n_map = 4
        self.n_action = 4
        self.gamma = 0.9
        self.grid_world = np.array([0,0,0,0,
                     0,-1,0,-1,
                     0,0,0,-1,
                    -1,0,0,1]).reshape(self.n_map,-1)
        self.value = np.zeros_like(self.grid_world)
    
    
    '''
    Evaluation
    '''
    def Evaluation(self):
        v_tmp = np.zeros_like(self.grid_world, dtype=np.float16)
        for row in range(self.n_map):
            for col in range(self.n_map):
                
                max_val = -9999
                for a in range(self.n_action):
                    reward = self.get_reward(a, row, col)
                    
                    # V.I 계산식 - bellman optimality equation.
                    next_state = self.get_state(a, row, col)
                    max_tmp = reward + self.gamma * self.value[next_state[0], next_state[1]]
                    if(max_tmp > max_val):
                        max_val = max_tmp
                        
                v_tmp[row, col] = max_val
        
        v_tmp[3,3] = 0
        self.value = v_tmp
        

    '''
    R(s,a)
    P(s'|s,a)
    
    0  1  2  3
    상 하 좌 우
    '''
    def get_reward(self, a, r, c):
        
        if(a == 0 and r > 0):
            r -= 1

        elif(a == 1 and r < 3):
            r += 1

        elif(a == 2 and c > 0):
            c -= 1

        elif(a == 3 and c < 3):
            c += 1

        reward = self.grid_world[r, c]

        return reward

    
    '''
    next state
    '''
    def get_state(self, a, r, c):
        
        if(a == 0 and r > 0):
            r -= 1

        elif(a == 1 and r < 3):
            r += 1

        elif(a == 2 and c > 0):
            c -= 1

        elif(a == 3 and c < 3):
            c += 1
            
        return r, c

In [32]:
v_i = Value_Iteration()

In [36]:
for i in range(10):
    v_i.Evaluation()

print(v_i.grid_world)
print(v_i.value)

[[ 0  0  0  0]
 [ 0 -1  0 -1]
 [ 0  0  0 -1]
 [-1  0  0  1]]
[[0.591  0.6562 0.729  0.6562]
 [0.6562 0.729  0.81   0.729 ]
 [0.729  0.81   0.9    1.    ]
 [0.81   0.9    1.     0.    ]]
